# TOC
1. Importing library & main data   
2. Matching county names to JSON format   
3. Importing county FIPS data   
4. Merging FIPS data into main dataframe   
5. Separating address information into new dataframe   
6. Exporting unique addresses   
7. Importing store latitude/longitude data   
8. Merging latitude/longitude data into main dataframe   

In [1]:
# Importing libraries    
import pandas as pd
import numpy as np
import os

In [2]:
# Importing data
path = r'C:\Users\Ryzen RGB Madness!!!\Iowa Liquor Sales Analysis'

In [3]:
liquor_data_complete = pd.read_pickle(os.path.join(path, '01 - Data', 'Cleaned', 'iowa_liquor_sales_and_population.pkl'))

In [4]:
# Setting county names into proper capitalization to match JSON data
liquor_data_complete['county'] = liquor_data_complete['county'].str.title()

In [5]:
# Matching O'Brien County to FIPS & JSON data
liquor_data_complete['county'] = liquor_data_complete['county'].str.replace("O'Brien","Obrien")

In [6]:
liquor_data_complete.head()

,invoice_number,date,store_number,store_name,address,city,zip_code,county,population,category,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-14210900001,2018-09-04,4617,LICKETY LIQUOR,2501 HUBBELL AVE,DES MOINES,50317,Polk,497254.0,1031100.0,...,36904,MCCORMICK VODKA PET,24,375,1.80,2.70,48,129.60,18.0,4.75
1,INV-16812800013,2019-01-08,5151,IDA LIQUOR,500 HWY 175,IDA GROVE,51445,Ida,6960.0,1031100.0,...,38176,TITOS HANDMADE VODKA,12,750,9.64,14.46,12,173.52,9.0,2.37
2,INV-18087600031,2019-03-12,5151,IDA LIQUOR,500 HWY 175,IDA GROVE,51445,Ida,6960.0,1011600.0,...,27102,TEMPLETON 4YR RYE,6,750,18.09,27.14,6,162.84,4.5,1.18
3,INV-15226900023,2018-10-23,5151,IDA LIQUOR,500 HWY 175,IDA GROVE,51445,Ida,6960.0,1031100.0,...,35918,FIVE O'CLOCK VODKA,6,1750,7.20,10.80,6,64.80,10.5,2.77
4,INV-17301000008,2019-02-01,2445,RUBACK'S FOOD CENTER,504 SOUTH HIGHWAY,OAKLAND,51560,Pottawattamie,93426.0,1031100.0,...,36306,HAWKEYE VODKA,12,750,3.34,5.01,12,60.12,9.0,2.37


In [7]:
# Importing FIPS data
fips_data = pd.read_csv(os.path.join(path, '01 - Data', 'Original', 'iowa_county_fips.csv'))

In [8]:
fips_data.head()

,county_fips,county
0,19119,Lyon
1,19143,Osceola
2,19059,Dickinson
3,19063,Emmet
4,19005,Allamakee


In [9]:
# Merging FIPS data into main dataframe
liquor_data_complete = liquor_data_complete.merge(fips_data[['county_fips','county']],left_on=['county'],right_on=['county'],how='left')

In [10]:
liquor_data_complete.columns

Index(['invoice_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'county', 'population', 'category', 'category_name',
       'vendor_number', 'vendor_name', 'item_number', 'item_description',
       'pack', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'bottles_sold', 'sale_dollars', 'volume_sold_liters',
       'volume_sold_gallons', 'county_fips'],
      dtype='object')

In [11]:
# Nulls?
liquor_data_complete.isnull().sum()

invoice_number         0
date                   0
store_number           0
store_name             0
address                0
city                   0
zip_code               0
county                 0
population             0
category               0
category_name          0
vendor_number          0
vendor_name            0
item_number            0
item_description       0
pack                   0
bottle_volume_ml       0
state_bottle_cost      0
state_bottle_retail    0
bottles_sold           0
sale_dollars           0
volume_sold_liters     0
volume_sold_gallons    0
county_fips            0
dtype: int64

In [12]:
fips_nulls = liquor_data_complete[liquor_data_complete['county_fips'].isnull()==True]

In [13]:
fips_nulls.head()

,invoice_number,date,store_number,store_name,address,city,zip_code,county,population,category,...,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,county_fips


In [14]:
# Moving fips column before county 
liquor_data_complete = liquor_data_complete[['invoice_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'county_fips', 'county', 'population', 'category', 'category_name',
       'vendor_number', 'vendor_name', 'item_number', 'item_description',
       'pack', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'bottles_sold', 'sale_dollars', 'volume_sold_liters',
       'volume_sold_gallons']]

In [15]:
# Duplicating dataframe (lat/long prep)
addresses = liquor_data_complete[['address', 'city', 'zip_code']].copy()

In [16]:
addresses.shape

(12537543, 3)

In [17]:
# Unique addresses only
unique_addys = pd.DataFrame(addresses.address.dropna().unique(), columns=['address'])

In [18]:
unique_addys.shape

(2442, 1)

In [19]:
# exporting unique addresses to csv
unique_addys.to_csv(os.path.join(path, '01 - Data', 'Cleaned', 'iowa_store_lats_longs.csv'))

#### Outside Python:
1. Verified no duplicates (Excel)   
2. Added state column (Excel)   
3. Cleaned unnecessary information from address column, notated row and what was (temporarily) removed (Excel)   
4. Uploaded spreadsheet to Geocodio to get latitude/longitude data (Geocodio)   
5. Saved original Geocodio csv with all data (including accuracy and source)   
6. Removed unnecessary columns that don't need to be appended into dataset (Excel)   
7. Saved revised csv as iowa_data_lats_longs

In [20]:
# Importing lat/long data
latlong_data = pd.read_csv(os.path.join(path, '01 - Data', 'Cleaned', 'iowa_data_lats_longs.csv'))

In [21]:
latlong_data.shape

(2442, 3)

In [22]:
latlong_data.head()

,address,latitude,longitude
0,2501 HUBBELL AVE,41.607739,-93.570182
1,500 HWY 175,42.350164,-95.489235
2,504 SOUTH HIGHWAY,41.309302,-95.396649
3,ONE PRAIRIE MEADOWS DRIVE,41.654455,-93.489788
4,910 N JEFFERSON,41.368740,-93.556483


In [23]:
# Merging lat/long data into main dataframe
liquor_data_all = liquor_data_complete.merge(latlong_data[['address', 'latitude', 'longitude']],left_on=['address'],right_on=['address'],how='left')

In [24]:
liquor_data_all.shape

(12537543, 26)

In [25]:
liquor_data_all.head()

,invoice_number,date,store_number,store_name,address,city,zip_code,county_fips,county,population,...,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,latitude,longitude
0,INV-14210900001,2018-09-04,4617,LICKETY LIQUOR,2501 HUBBELL AVE,DES MOINES,50317,19153,Polk,497254.0,...,24,375,1.80,2.70,48,129.60,18.0,4.75,41.607739,-93.570182
1,INV-16812800013,2019-01-08,5151,IDA LIQUOR,500 HWY 175,IDA GROVE,51445,19093,Ida,6960.0,...,12,750,9.64,14.46,12,173.52,9.0,2.37,42.350164,-95.489235
2,INV-18087600031,2019-03-12,5151,IDA LIQUOR,500 HWY 175,IDA GROVE,51445,19093,Ida,6960.0,...,6,750,18.09,27.14,6,162.84,4.5,1.18,42.350164,-95.489235
3,INV-15226900023,2018-10-23,5151,IDA LIQUOR,500 HWY 175,IDA GROVE,51445,19093,Ida,6960.0,...,6,1750,7.20,10.80,6,64.80,10.5,2.77,42.350164,-95.489235
4,INV-17301000008,2019-02-01,2445,RUBACK'S FOOD CENTER,504 SOUTH HIGHWAY,OAKLAND,51560,19155,Pottawattamie,93426.0,...,12,750,3.34,5.01,12,60.12,9.0,2.37,41.309302,-95.396649


In [28]:
# Moving lat/long columns before county_fips
liquor_data_all = liquor_data_all[['invoice_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'latitude', 'longitude', 'county_fips', 'county', 'population', 'category', 'category_name',
       'vendor_number', 'vendor_name', 'item_number', 'item_description',
       'pack', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'bottles_sold', 'sale_dollars', 'volume_sold_liters',
       'volume_sold_gallons']]

In [29]:
# Checking for nulls
liquor_data_all.isnull().sum()

invoice_number         0
date                   0
store_number           0
store_name             0
address                0
city                   0
zip_code               0
latitude               0
longitude              0
county_fips            0
county                 0
population             0
category               0
category_name          0
vendor_number          0
vendor_name            0
item_number            0
item_description       0
pack                   0
bottle_volume_ml       0
state_bottle_cost      0
state_bottle_retail    0
bottles_sold           0
sale_dollars           0
volume_sold_liters     0
volume_sold_gallons    0
dtype: int64

In [32]:
# Exporting full dataset
liquor_data_all.to_pickle(os.path.join(path, '01 - Data', 'Cleaned', 'iowa_sales_location_population.pkl'))